In this notebook, I tried training a model to predict a bounding box from an MRI slice rather than just instantly predicting the pixelwise segmentation. 
It didn't seem too great and was definitely beat by the yolov7 model I trained.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
join = os.path.join
from tqdm import tqdm
from skimage import transform
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
import monai
import sys
sys.path.append('./modified_medsam_repo')
from segment_anything import sam_model_registry
import torch.nn.functional as F
import argparse
import random
from datetime import datetime
import shutil
import glob
import pandas as pd
import nibabel as nib
import pickle
import time

from MedSAM_HCP.dataset import MRIDataset, load_datasets
from MedSAM_HCP.MedSAM import MedSAM, medsam_inference
from MedSAM_HCP.build_sam import build_sam_vit_b_multiclass
from MedSAM_HCP.utils_hcp import *
from MedSAM_HCP.loss_funcs_hcp import *

In [2]:
def proc_arr(arr):
    # arr has shape (classes, *)
    ax_starts = np.argmax(arr, axis=1) # shape (classes)
    ax_ends = arr.shape[1] - 1 - np.argmax(arr[:,::-1], axis=1) # shape (classes)

    maxs = np.max(arr, axis = 1) # shape (classes)
    ax_starts = np.where(maxs == 1, ax_starts, np.nan)
    ax_ends = np.where(maxs == 1, ax_ends, np.nan)

    return ax_starts, ax_ends


def get_bounding_box(seg_tens):
    # seg_tens has shape (256,256,256)
    # return shape (4, classes) - rmin, rmax, cmin, cmax
    
    cols = np.any(seg_tens, axis=1) # (classes, W)
    rows = np.any(seg_tens, axis=2) # (classes, H) of True/False, now find min row and max row with True
    
    rmin, rmax = proc_arr(rows)
    cmin, cmax = proc_arr(cols)
    
    return np.array((rmin, rmax, cmin, cmax))

def conv_format(seg_tens):

    # return shape (num_classes, 4) - x, y, width, height
    if type(seg_tens) == torch.Tensor:
        seg_tens = seg_tens.cpu().detach().numpy()
    res = get_bounding_box(seg_tens) # 4 x num_classes - rmin, rmax, cmin, cmax

    return res # 4 x num_classes

In [3]:
class MRIDataset(Dataset): 
    def __init__(self, data_frame, label_converter=None, NUM_CLASSES = 256):
        self.data_frame = data_frame
        self.label_converter = label_converter
        self.NUM_CLASSES = NUM_CLASSES

        if self.label_converter is None:
            print('Initializing with no label converter, are you sure the labels are correct?')

        #print(f'number of images: {data_frame.shape[0]}')
    
    def __len__(self):
        return self.data_frame.shape[0]

    def __getitem__(self, index):
        # load image embedding as npy
        img_embed_path = self.data_frame.loc[index,'image_embedding_slice_path']
        img_embed_npy = np.load(img_embed_path) # (256, 64, 64)
        img_slice_name = '_slice'.join(img_embed_path.split('/')[-2:])

        # load segmentation mask as npy
        seg_path = self.data_frame.loc[index,'segmentation_slice_path']
        seg_npy = np.load(seg_path) # (256, 256)

        if self.label_converter is not None:
            #print(f'pre label max: {seg_npy.max()}')
            seg_npy = self.label_converter.hcp_to_compressed(seg_npy)
            #print(f'post label max: {seg_npy.max()}')
            
        seg_tens = torch.LongTensor(seg_npy[None, :, :]) # B, H, W
        seg_tens = torch.nn.functional.one_hot(seg_tens, num_classes=self.NUM_CLASSES) # B, H, W, C
        
        seg_tens = torch.permute(seg_tens, (0, 3, 1, 2)) # B, C, H, W
        seg_tens = seg_tens[0] # exclude batch dimension -> C, H, W

        bboxes = conv_format(seg_tens).T # (C,4)
        is_present = np.invert(np.isnan(bboxes[:, 0:1])) # (C)
        bboxes[np.invert(is_present.flatten()), :] = 0

        outputs = np.hstack((bboxes, is_present))


        return torch.tensor(img_embed_npy).float(), torch.tensor(outputs).int(), img_slice_name
    
    def load_image(self, index):
        img_path = self.data_frame.loc[index, 'image_path']
        slice_num = self.data_frame.loc[index, 'slice']
        img = nib.load(img_path).get_fdata()[:,slice_num,:].astype(np.uint8)
        return img # returns as (256, 256)

In [4]:
def load_datasets(path_df_path, train_test_splits_path, sample_n_slices = None, label_converter=None, NUM_CLASSES=256):
    # load dataframe of slice paths
    path_df = pd.read_csv(path_df_path)

    # load train val test ids
    dicto = pickle.load(open(train_test_splits_path, 'rb'))
    train_ids = dicto['train']
    val_ids = dicto['val']
    test_ids = dicto['test']

    train_df = path_df[path_df['id'].isin(train_ids)].reset_index(drop=True)
    val_df = path_df[path_df['id'].isin(val_ids)].reset_index(drop=True)
    test_df = path_df[path_df['id'].isin(test_ids)].reset_index(drop=True)

    if sample_n_slices is not None:
        train_df = train_df.sample(n=sample_n_slices, replace=False, random_state=2023).reset_index(drop=True)
        val_df = val_df.sample(n=sample_n_slices, replace=False, random_state=2023).reset_index(drop=True)
        test_df = test_df.sample(n=sample_n_slices, replace=False, random_state=2023).reset_index(drop=True)
    train_dataset = MRIDataset(train_df, label_converter = label_converter, NUM_CLASSES=NUM_CLASSES)
    val_dataset = MRIDataset(val_df, label_converter = label_converter, NUM_CLASSES=NUM_CLASSES)
    test_dataset = MRIDataset(test_df, label_converter = label_converter, NUM_CLASSES=NUM_CLASSES)

    return train_dataset, val_dataset, test_dataset

In [35]:
class BBoxPredictor(nn.Module):
    def __init__(self, num_class):
        super().__init__()
        self.flatten = nn.Flatten()
        self.lin1 = nn.Linear(256*64*64, 5 * num_class)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.num_class = num_class
    def forward(self, image_embedding):
        
        # image_embedding: shape (B, 256, H, W)
        # output should be (B, num_class, 5)
        B, F, H, W = image_embedding.shape
        flattened = self.flatten(image_embedding)
        z = self.lin1(flattened) # (B, 5*num_class)
        z = z.view(B, self.num_class, 5)

        z[:, :, :4] = self.relu(z[:, :, :4])
        #z[:, :, 4] = self.sigmoid(z[:, :, 4])

        return z

def loss_fn(pred, gt):
    # pred has shape (B, C, 5)
    # gt has shape (B, C, 5)
    
    # loss term for pred probability
    # and if ground truth is 1, also loss term for bounding box values

    binary_loss = nn.BCEWithLogitsLoss(reduction = 'mean')
    regression_loss = nn.MSELoss(reduction='none')
    
    binary_part = binary_loss(pred[:, :, 4], gt[:, :, 4].float())
    regression_part = regression_loss(pred[:, :, :4], gt[:, :, :4].float()) / 4096
    regression_part = (regression_part * gt[:, :, 4:5]).sum(dim=2).mean() # shape: (B, C, 4)
    loss = binary_part + regression_part

    print(f'binary: {binary_part}')
    print(f'regression: {regression_part}')
    return loss



In [6]:
df_hcp = pd.read_csv('/gpfs/home/kn2347/MedSAM/hcp_mapping_processed.csv')
df_desired = pd.read_csv('/gpfs/home/kn2347/MedSAM/darts_name_class_mapping_processed.csv')
NUM_CLASSES = len(df_desired)
label_converter = LabelConverter(df_hcp, df_desired)
train_dataset, val_dataset, test_dataset = load_datasets('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_constant_bbox.csv', '/gpfs/data/luilab/karthik/pediatric_seg_proj/train_val_test_split.pickle', sample_n_slices = None, label_converter=label_converter, NUM_CLASSES=NUM_CLASSES)


train_dataloader = DataLoader(
    train_dataset,
    batch_size = 32,
    shuffle = True,
    num_workers = 8,
    pin_memory = True,
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size = 32,
    shuffle = False,
    num_workers = 1,
    pin_memory = True,
)

/gpfs/home/kn2347/.conda/envs/medsam/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [36]:
model = BBoxPredictor(num_class = NUM_CLASSES).cuda()
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=.01
)
for epoch in range(5):
    for step, (image_embedding, bbox, _) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        image_embedding, bbox = image_embedding.cuda(), bbox.cuda()
        output = model(image_embedding)
        loss = loss_fn(output, bbox)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()



  0%|          | 1/7128 [00:14<27:53:56, 14.09s/it]

binary: 0.6941751837730408
regression: 0.621527910232544
binary: 0.9802172183990479
regression: 0.7128661274909973


  0%|          | 5/7128 [00:14<3:39:25,  1.85s/it] 

binary: 0.7561362385749817
regression: 0.5468178987503052
binary: 0.6672207713127136
regression: 0.508444607257843
binary: 0.43888136744499207
regression: 0.4834926724433899


  0%|          | 7/7128 [00:14<2:13:24,  1.12s/it]

...

binary: 0.030206570401787758
regression: 0.11396850645542145


  7%|▋         | 474/7128 [09:51<2:36:24,  1.41s/it]

binary: 0.06536858528852463
regression: 0.2376554012298584


  7%|▋         | 474/7128 [09:52<2:18:35,  1.25s/it]


KeyboardInterrupt: 